In [5]:
from langchain_community.llms import Ollama
import time

In [59]:
model = Ollama(model="llama3")
def run(chain,prompt=""): 
    tokens = ""
    start = time.time()
    for chunk in chain.stream(prompt): 
        print(chunk, end='', flush=True)
        tokens += chunk
    end = time.time()
    print(f"\n\n => tokens/second : {len(tokens.split(" "))/(end-start)}")

In [64]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [47]:
prompt = PromptTemplate.from_template(
    """
    You are a Calssifer assistant that helps me 
    classify customers feedback as Good, Bad, Neutral or Don't know 
    please only stick to these options and only output one word 
    nothing more nothing less 
    I want you to classify this : 
    {feedback}
    """
)
parser = StrOutputParser()

classifier = prompt | model | parser

In [63]:
good_chain = (lambda x: "write a possitive message with emoji's") | model | parser
bad_chain = (lambda x: "write a negative message with emoji's") | model | parser
neutral_chain = (lambda x: "write a Neutral message with emoji's") | model | parser

In [ ]:
router_chain = RunnableBranch(
    (
        lambda x: "Good" in x,
        good_chain
    ),
    (
        lambda x: "Bad" in x,
        bad_chain
    ),
    (
        lambda x: "Neutral" in x,
        neutral_chain
    ),
)